In [ ]:
%matplotlib inline
import yt
import logging
logging.getLogger('yt').setLevel(logging.ERROR)
import numpy as np
import matplotlib
matplotlib.rcParams['savefig.dpi'] = 150
import matplotlib.pyplot as plt
import multiprocessing
import sys
sys.path.append('/home/ychen/lib/util')
import util
#dss = yt.load('/home/ychen/d9/FLASH4/stampede/0529_L45_M10_b1_h1/MHD_Jet_hdf5_plt_cnt*')

In [ ]:
def rescan(dir, printlist=False):
    files = util.scan_files(dir, '*hdf5_plt_cnt_[0-9][0-9][0-9][0-9]', \
                             printlist=printlist, walk=True)
    return files

dir = '/home/ychen/d9/FLASH4/stampede/0529_L45_M10_b1_h1/'
files = rescan(dir, printlist=True)

In [ ]:
ds = yt.load(files[100].fullpath)
ad = ds.all_data()
zlim = (ds.domain_left_edge[2]/8.0, ds.domain_right_edge[2]/8.0)
print zlim[0].in_units('kpc')

In [ ]:
prof = yt.create_profile(ad, 'z', ['dens'], extrema={'z': zlim}, n_bins=20, logs={'z': False})

In [ ]:
plt.plot(prof.x, prof['dens'])

In [ ]:
#sp = ds.sphere([0.0,0.0,0.0], (60, 'kpc'))
prof = yt.create_profile(ad, 'z', ['jet ', 'pressure', 'temperature'], \
                         logs={'z': False}, weight_field=None, extrema={'z': zlim})
prof_wf = yt.create_profile(ad, 'z', ['jet ', 'pressure', 'temperature'], \
                         logs={'z': False}, weight_field='cell_mass', extrema={'z': zlim})

In [ ]:
fig, ax1 = plt.subplots()
ax1.plot(prof.x, prof['jet '], c='r', label='pressure')
ax1.plot(prof_wf.x, prof_wf['jet '], c='b', label='pressure')
ax1.semilogy()
ax1.set_ylim(1E-11, 1E6)

In [ ]:
for (z, j) in reversed(zip(prof.x, prof['jet '])):
    if j > 1E4:
        print z, j
        break

In [ ]:
dirs = [
        '/home/ychen/data/0only_0605_h0/',\
        '/home/ychen/data/0only_0602_hydro/',\
        '/home/ychen/data/0only_0529_h1/',\
        '/home/ychen/data/0only_0605_hinf/',\
        '/home/ychen/data/0only_0204_h0_10Myr/',\
        '/home/ychen/data/0only_0413_hydro_10Myr/',\
        '/home/ychen/data/0only_1022_h1_10Myr/',\
        '/home/ychen/data/0only_0204_hinf_10Myr/']

colors = {}
labels = {}
for dirname in dirs:
    if 'h1' in dirname:
        colors[dirname] = 'r'
        labels[dirname] = 'helical'
    elif 'h0' in dirname:
        colors[dirname] = 'b'
        labels[dirname] = 'poloidal'
    elif 'hinf' in dirname:
        colors[dirname] = 'g'
        labels[dirname] = 'toroidal'
    else:
        colors[dirname] = 'grey'
        labels[dirname] = 'hydro'

for dir in dirs:
    data = np.loadtxt(dir+'GridAnalysis_LobeSize.txt')
    label = labels[dir]
    if '10Myr' in dir:
        plt.fill_between(data[:,1], data[:,2], -data[:,3],\
                         facecolor=colors[dir], lw=1, edgecolor=colors[dir], alpha=0.7)
        #plt.plot(data[:,1], data[:,2], c=colors[dir])
    else:
        plt.fill_between(data[:,1], data[:,2], -data[:,3],\
                         facecolor=colors[dir], lw=1, edgecolor=colors[dir], alpha=0.7, label=label)
    plt.fill_between(data[:,1], data[:,4], -data[:,5],\
                     facecolor=colors[dir], lw=1, linestyle=':', edgecolor=colors[dir], alpha=0.3)
plt.legend(loc=4)
plt.ylabel('z (kpc)')
plt.xlabel('t (Myr)')
plt.xlim(0,100)
#plt.semilogx()

In [ ]:
dirs = [
        '/home/ychen/data/0only_0529_h1/',\
        '/home/ychen/data/0only_0605_hinf/',\
        '/home/ychen/data/0only_0602_hydro/',\
        '/home/ychen/data/0only_0605_h0/']

colors = {}
for dirname in dirs:
    if 'h1' in dirname:
        colors[dirname] = 'r'
        labels[dirname] = 'helical'
    elif 'h0' in dirname:
        colors[dirname] = 'b'
        labels[dirname] = 'poloidal'
    elif 'hinf' in dirname:
        colors[dirname] = 'g'
        labels[dirname] = 'toroidal'
    elif 'hydro' in dirname:
        colors[dirname] = 'grey'
        labels[dirname] = 'hydro'

for dir in dirs:
    data = np.loadtxt(dir+'GridAnalysis_LobeSize.txt')
    #label = dir[28:-1]
    label = labels[dir]
    plt.fill_between(data[:,1], data[:,2], -data[:,3],\
                     facecolor=colors[dir], lw=1, edgecolor=colors[dir], alpha=0.8, label=label)
    plt.fill_between(data[:,1], data[:,4], -data[:,5],\
                     facecolor=colors[dir], lw=1, linestyle=':', edgecolor=colors[dir], alpha=0.3)

plt.legend(loc=4)
plt.ylabel('z (kpc)')
plt.xlabel('t (Myr)')
#plt.axvline(10, ls=':', c='grey')

In [ ]:
#upfilter = ad['z'] > 0.0
kpc = yt.units.kpc.in_units('cm')
leftedge = [-30*kpc, -30*kpc, 0.0]
rightedge = [ 30*kpc, 30*kpc, 60*kpc]
upbox = ds.box(leftedge, rightedge)

In [ ]:
np.sum(upbox['jet ']*upbox['cell_mass']*upbox['z'])/np.sum(upbox['jet ']*upbox['cell_mass'])

In [ ]:
fig, ax1 = plt.subplots()
ax1.plot(prof_p.x, prof_p['pressure'], c='b', label='pressure')
plt.legend(loc=2)
ax2 = ax1.twinx()
ax2.plot(prof_t.x, prof_t['temperature'], c='r', label='temperature')
plt.legend(loc=1)
#plot = yt.ProfilePlot.from_profiles([prof_p, prof_t], labels=['pressure', 'temperature'])

In [ ]:
plt.plot(prof[0].x[1:], (prof[0]['temperature'][1:]-prof[0]['temperature'][0:-1])/)